In [1]:
import os
os.sys.path.append('..')

In [2]:
import warnings
warnings.simplefilter("ignore")

# autoreload import your package
%load_ext autoreload
%autoreload 2

In [3]:
from loguru import logger
logger.remove()
logger.add(os.sys.stdout, level="ERROR", colorize=True, format="<level>{time} | {message}</level>")
# import_dir(ta_dir, verbose=False)
warnings.simplefilter("ignore")

In [4]:
import torch
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

# Args

In [5]:
from run import set_seed, get_args, Exp_Main

In [ ]:
# mimic cli args to avoid code duplication
argv = """python -u run.py \
  --root_path ../dataset/stocks/ \
  --data_path OXY_2019.csv.gz \
  --checkpoints ./checkpoints/ \
  --model_id Exchange \
  --model ETSformer \
  --data custom \
  --features S \
  --seq_len 128 \
  --pred_len 48 \
  --e_layers 2 \
  --d_layers 2 \
  --enc_in 1 \
  --c_out 1 \
  --num_workers 0 \
  --des 'Exp' \
  --K 0 \
  --learning_rate 1e-3 \
  --target RSMKs_18_144_72_2ref_2ref \
  --itr 1
"""
argv = argv.replace("\\n", "").split()[3:]
args = get_args(argv)
args


Exp = Exp_Main
ii=0
set_seed(ii)
# setting record of experiments
setting = '{}_{}_{}_ft{}_sl{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_K{}_lr{}_{}_{}'.format(
    args.model_id,
    args.model,
    args.data,
    args.features,
    args.seq_len,
    args.pred_len,
    args.d_model,
    args.n_heads,
    args.e_layers,
    args.d_layers,
    args.d_ff,
    args.K,
    args.learning_rate,
    args.des, ii)

#     if os.path.exists(os.path.join(args.checkpoints, setting)):
#         print('skipping exists')
#         continue

exp = Exp(args)  # set experiments
print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
exp.train(setting)

print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
exp.test(setting, data='val')
exp.test(setting, data='test')

torch.cuda.empty_cache()

Args in experiment:
Namespace(K=0, activation='sigmoid', batch_size=32, c_out=1, checkpoints='./checkpoints/', d_ff=2048, d_layers=2, d_model=512, damping_learning_rate=0, data='custom', data_path='OXY_2019.csv.gz', dec_in=7, des="'Exp'", devices='0,1,2,3', dropout=0.2, e_layers=2, embed='timeF', enc_in=1, features='S', freq='h', gpu=0, itr=1, label_len=0, learning_rate=0.001, lradj='exponential_with_warmup', min_lr=1e-30, model='ETSformer', model_id='Exchange', n_heads=8, num_workers=0, optim='adam', output_attention=False, patience=5, pred_len=48, root_path='../dataset/stocks/', seq_len=128, smoothing_learning_rate=0, std=0.2, target='RSMKs_18_144_72_2ref_2ref', train_epochs=15, use_gpu=True, use_multi_gpu=False, warmup_epochs=3)
Use GPU: cuda:0
>>>>>>>start training : Exchange_ETSformer_custom_ftS_sl128_pl48_dm512_nh8_el2_dl2_df2048_K0_lr0.001_'Exp'_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 37203
val 5294
test 10632


  0%|          | 0/15 [00:00<?, ?epoch/s]

train:   0%|          | 0/1162 [00:00<?, ?it/s]

# Plot

In [ ]:
setting

In [ ]:
ds, dl = exp._get_data('test')

In [ ]:
preds, trues = exp.test(setting, data='test')

In [ ]:
i=990
(batch_x, batch_y, batch_x_mark, batch_y_mark)  = ds[i]
l1 = batch_x.shape[0]
l2 = batch_y.shape[0]
plt.plot(range(l1), batch_x[:, -1], label='past')
plt.plot(range(l1, l1+l2), batch_y[:, -1], color='blue', ls='--', label='true')
plt.plot(range(l1, l1+l2), preds[i], label='pred');
plt.legend(loc='lower left')

# TODO check index

- s_end = index + self.seq_len
- r_begin = index + self.seq_len - self.label_len
- r_end = index + self.seq_len + self.pred_len

In [ ]:
f = ds.root_path + '/' + ds.data_path
df = pd.read_csv(f).set_index('date', drop=False)
df = df[ds.cols[1:]]
# df[:] = ds.scaler.transform(df.values)
df
# df

In [ ]:
from data_provider.data_loader import Dataset_Custom
ds2 = Dataset_Custom(
    ds.root_path,
    flag="test",
#     size=ds.size,
    size=[ds.seq_len, ds.label_len, ds.pred_len],
    features=ds.features,
    data_path=ds.data_path,
    target=ds.target,
    scale=False,
    timeenc=ds.timeenc,
    freq=ds.freq
)

i=99
(batch_x, batch_y, batch_x_mark, batch_y_mark)  = ds2[i]
batch_x[-1]

In [ ]:
dt = ds2.index.iloc[i-1]
df.loc[dt]

In [ ]:
assert df.loc[dt].close == batch_x[-1, 0], 'index should be right'
'OK'